<a href="https://colab.research.google.com/github/daini10421/Mini-Project-54-Sentiment-Analysis-on-Hospital-Comments/blob/main/Mini_Project_54_Sentiment_Analysis_on_Hospital_Comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis
- Private dataset with sensitive hospital comments

# Importing Packages

In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
import spacy
import en_core_web_sm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.

In [ ]:
df = pd.read_csv("/kaggle/input/private-hospital-comments/comments1.csv")

# Preprocessing the Dataset

In [ ]:
df["Main Topic"] = df["Main Topic"].fillna("No Topic Given")
df = df.dropna(subset=["Content"])
nan_counts = df.isna().sum()

print(nan_counts)

In [ ]:
# Defining regex patterns.
linebreaks        = "<br /><br />"
alphaPattern      = "[^a-z0-9<>]"
sequencePattern   = r"(.)\1\1+"
seqReplacePattern = r"\1\1"

# Defining regex for emojis
smileemoji        = r"[8:=;]['`\-]?[)d]+"
sademoji          = r"[8:=;]['`\-]?\(+"
neutralemoji      = r"[8:=;]['`\-]?[\/|l*]"
lolemoji          = r"[8:=;]['`\-]?p+"

stop_words = set(stopwords.words('english'))
Lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_reviews(review):

    review = review.lower()

    review = re.sub(linebreaks," ",review)
    # Replace 3 or more consecutive letters by 2 letter.
    review = re.sub(sequencePattern, seqReplacePattern, review)

    # Replace all emojis.
    review = re.sub(r'<3', '<heart>', review)
    review = re.sub(smileemoji, '<smile>', review)
    review = re.sub(sademoji, '<sadface>', review)
    review = re.sub(neutralemoji, '<neutralface>', review)
    review = re.sub(lolemoji, '<lolface>', review)

    # Remove non-alphanumeric and symbols
    review = re.sub(alphaPattern, ' ', review)

    # Tokenize the input text
    tokens = word_tokenize(review)

    # Remove stop words from the token sequence

    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the remaining tokens
    tokens = [Lemmatizer.lemmatize(token) for token in tokens]

    # Join the cleaned tokens into a single string
    return ' '.join(tokens)


In [ ]:
# combine "Main Topic", "Subtopic", and "Content" columns into a single column called "Text"
df["Text"] = df["Main Topic"] + " " + df["Subtopic"] + " " + df["Content"]

In [ ]:
df["Text"] = df["Text"].apply(preprocess_reviews)

# Keyword and Sentiment Analysis using Word2Vec
- Reference Article: https://towardsdatascience.com/unsupervised-semantic-sentiment-analysis-of-imdb-reviews-2c5f520fbf81

In [ ]:
!curl https://raw.githubusercontent.com/towardsNLP/IMDB-Semantic-Sentiment-Analysis/main/Word2Vec/src/w2v_utils.py -o w2v_utils.py

In [ ]:
from w2v_utils import (Tokenizer,
                       evaluate_model,
                       bow_vectorizer,
                       train_logistic_regressor,
                       w2v_trainer,
                       calculate_overall_similarity_score,
                       overall_semantic_sentiment_analysis,
                       list_similarity,
                       calculate_topn_similarity_score,
                       topn_semantic_sentiment_analysis,
                       define_complexity_subjectivity_reviews,
                       explore_high_complexity_reviews,
                       explore_low_subjectivity_reviews,
                       text_SSA)

In [ ]:

# Instancing the Tokenizer class
tokenizer = Tokenizer(clean= True,
                      lower= True,
                      de_noise= True,
                      remove_stop_words= True,
                      keep_negation=True)


In [ ]:
df['tokenized_text'] = df['Text'].apply(tokenizer.tokenize)

df['tokenized_text_len'] = df['tokenized_text'].apply(len)
df['tokenized_text_len'].apply(np.log).describe()

In [ ]:
keyed_vectors , keyed_vocab = w2v_trainer(df["tokenized_text"])

In [ ]:
print(type(keyed_vectors))
print(type(keyed_vocab))

In [ ]:
keyed_vectors.most_similar("research",topn=15)

In [ ]:
keyed_vectors.most_similar("hospital",topn=15)

In [ ]:
keyed_vectors.most_similar("funded",topn=15)

# Clustering Approach to Sentiment Analysis

In [ ]:
# To make sure that all `positive_concepts` are in the keyed word2vec vocabulary
positive_concepts = ['excellent', 'awesome', 'cool','decent','amazing', 'strong', 'good', 'great', 'funny', 'entertaining']
pos_concepts = [concept for concept in positive_concepts if concept in keyed_vocab]

In [ ]:
# To make sure that all `negative_concepts` are in the keyed word2vec vocabulary
negative_concepts = ['terrible','awful','horrible','boring','bad', 'disappointing', 'weak', 'poor',  'senseless','confusing']
neg_concepts = [concept for concept in negative_concepts if concept in keyed_vocab]


In [ ]:
# Calculating Semantic Sentiment Scores by OSSA model
overall_df_scores = overall_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   positive_target_tokens = pos_concepts,
                                                   negative_target_tokens = neg_concepts,
                                                   doc_tokens = df['tokenized_text'])

# Calculating Semantic Sentiment Scores by TopSSA model
topn_df_scores = topn_semantic_sentiment_analysis (keyed_vectors = keyed_vectors,
                                                   positive_target_tokens = pos_concepts,
                                                   negative_target_tokens = neg_concepts,
                                                   doc_tokens = df['tokenized_text'],
                                                     topn=30)


# To store semantic sentiment store computed by OSSA model in df
df['overall_PSS'] = overall_df_scores[0]
df['overall_NSS'] = overall_df_scores[1]
df['overall_semantic_sentiment_score'] = overall_df_scores[2]
df['overall_semantic_sentiment_polarity'] = overall_df_scores[3]



# To store semantic sentiment store computed by TopSSA model in df
df['topn_PSS'] = topn_df_scores[0]
df['topn_NSS'] = topn_df_scores[1]
df['topn_semantic_sentiment_score'] = topn_df_scores[2]
df['topn_semantic_sentiment_polarity'] = topn_df_scores[3]

In [ ]:
words = keyed_vectors.index_to_key
vectors = [keyed_vectors[word] for word in words]


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
result = pca.fit_transform(vectors)

# Create a DataFrame with PCA results and words
pca_df = pd.DataFrame(result, columns=['x', 'y'])
pca_df['word'] = words


In [ ]:
import plotly.graph_objs as go

fig = go.Figure(data=go.Scattergl(
    x=pca_df['x'],
    y=pca_df['y'],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        line_width=1
    ),
    text=pca_df['word'],
    textposition="bottom center"
))

fig.show()


In [ ]:
actual_pos_filt = df['topn_semantic_sentiment_polarity'] == 1
actual_neg_filt =  df['topn_semantic_sentiment_polarity'] == 0

# filter positive and negative review based on Most Probable predicted 'y' or 'topn_semantic_sentiment_score' column
predicted_pos_filt = df['topn_semantic_sentiment_polarity'] == 1
predicted_neg_filt = df['topn_semantic_sentiment_polarity'] == 0



# plotting Semantic Sentiment Score Position of Actual Negative Reviews
plt.scatter(df['topn_NSS'][actual_neg_filt],
         df['topn_PSS'][actual_neg_filt],
         label='Actual Negetive Reviews',
           color='DarkRed',
            alpha=0.4 , # set transparency of color
            s=20 # set size of dots
           )

# plotting Semantic Sentiment Score Position of Actual Positive Reviews
plt.scatter(df['topn_NSS'][actual_pos_filt],
         df['topn_PSS'][actual_pos_filt],
         label='Actual Positive Reviews',
       color='DarkGreen',
            alpha=0.1, # set transparency of color
            s=30 # set size of dots
           )
# naming the x & y axis
plt.xlabel('Predicted Negative Labels')
plt.ylabel('Predicted Positive Labels')

# Sentiment Analysis Using BERT

In [ ]:
!pip install transformers

In [ ]:

# importing the pipeline module
from transformers import pipeline

# Downloading the sentiment analysis model
SentimentClassifier = pipeline("sentiment-analysis")

# Downloading the sentiment analysis model
# SentimentClassifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


In [ ]:
# Defining a function to call for the whole dataframe
def FunctionBERTSentiment(inpText):
  return(SentimentClassifier(inpText)[0]['label'])

In [ ]:
df['BERT_Sentiment']=df['Text'].apply(FunctionBERTSentiment)
df.head(10)

In [ ]:
# Defining a function to call for the whole dataframe
def FunctionBERTScore(inpText):
  return(SentimentClassifier(inpText)[0]['score'])

In [ ]:
df['Score']=df['Text'].apply(FunctionBERTScore)
df.head(10)

In [ ]:
df['Content_BERT_Sentiment']=df['Content'].apply(FunctionBERTSentiment)
df.head(10)

In [ ]:
# df.to_csv('bert_sentiment.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
fig, subPlot =plt.subplots(nrows=1, ncols=2, figsize=(10,4))
fig.suptitle("Sentiment analysis of Content + Topic Text")

# Grouping the data
GroupedData=df.groupby('BERT_Sentiment').size()

# Creating the charts
GroupedData.plot(kind='bar', ax=subPlot[0], color=['crimson', 'lightblue'])
GroupedData.plot(kind='pie', ax=subPlot[1], colors=['crimson', 'lightblue'])

In [ ]:
import matplotlib.pyplot as plt
fig, subPlot =plt.subplots(nrows=1, ncols=2, figsize=(10,4))
fig.suptitle("Sentiment analysis of Content Only")

# Grouping the data
GroupedData=df.groupby('Content_BERT_Sentiment').size()

# Creating the charts
GroupedData.plot(kind='bar', ax=subPlot[0], color=['crimson', 'lightblue'])
GroupedData.plot(kind='pie', ax=subPlot[1], colors=['crimson', 'lightblue'])